# MNIST를 사용한 Linear Learner 소개
_**필기 숫자가 0인지에 대한 이진 예측 만들기**_

이 예제는 An Introduction to Linear Learner with MNIST의 한국어 번역본 입니다. 

https://github.com/awslabs/amazon-sagemaker-examples/blob/master/introduction_to_amazon_algorithms/linear_learner_mnist/linear_learner_mnist.ipynb

1. [소개](#소개)
2. [전제 조건 및 전처리](#전제-조건-및-전처리)
  1. [권한 및 환경 변수](#권한-및-환경-변수)
  2. [데이터 수집](#데이터-수집)
  3. [데이터 검사](#데이터-검사)
  4. [데이터 변환](#데이터-변환)
3. [선형 모델 훈련](#선형-모델-훈련)
4. [모델 호스팅 설정](#모델-호스팅-설정)
5. [사용하기위한 모델 검증](#사용하기위한-모델-검증)


## 소개
Amazon SageMaker의 Linear Learner을 소개하는 예제에 오신 것을 환영합니다. 오늘, 우리는 0부터 9까지의 필기체 숫자로 구성된 [MNIST](https://en.wikipedia.org/wiki/MNIST_database) 데이터셋을 분석해보겠습니다. 각 28 x 28 그레이 스케일 이미지의 개별 픽셀 값을 사용하여 이미지가 0부터 9까지 숫자에 대한 yes 또는 no 레이블을 예측합니다. 

우리가 사용할 방법은 선형 이진 분류기입니다. 선형 모델은 분류 또는 회귀 문제를 해결하는 데 사용되는 지도학습 알고리즘입니다. 입력으로는 모델에서 레이블이 지정된 예제 ( **`x`**, `y`)가 제공됩니다. **`x`** 는 고차원 벡터이며 `y` 는 숫자 레이블입니다. 이진 분류를 수행하기 때문에 알고리즘은 레이블이 0 또는 1 일 것으로 예상합니다(그러나 Amazon Sagemaker Linear Learner는 `y`의 연속 값에 대한 회귀도 지원합니다). 이 알고리즘은 벡터 **`x`** 를 레이블 `y`의 근사값에 매핑하여 선형 함수 또는 분류를 위한 선형 임계 값 함수를 학습합니다. 

Amazon SageMaker의 Linear Learner 알고리즘은 계산 효율적인 방식으로 많은 모델을 병렬로 학습하여 일반적인 선형 모델을 확장합니다. 각 모델에는 서로 다른 하이퍼파라메터 세트가 있으며 알고리즘은 특정 기준을 최적화하는 세트를 찾습니다. 이는 동일하거나 더 낮은 비용으로 일반적인 선형 알고리즘보다 훨씬 더 정확한 모델을 제공할 수 있습니다. 

시작하려면 권한, 구성 등을 위한 몇 가지 젠제 조건 단계로 환경을 설정해야 합니다. 


## 전제 조건 및 전처리

### 권한 및 환경 변수

_이 노트북은 ml.m4.xlarge 노트북 인스턴스에서 작성 및 테스트되었습니다._

다음을 지정하여 시작하겠습니다.

- 훈련 및 모델 데이터에 사용하려는 S3 버킷 및 접두사, 노트북 인스턴스, 훈련 및 호스팅은 동일 리전에 있어야 합니다. 
- 훈련 및 호스팅, 데이터 엑세스에 사용하는 IAM 역할 arn을 생성하는 방법에 대한 설명서를 참조하세요. 노트북 인스턴스, 훈련 및 호스팅에 두개 이상의 역할이 필요한 경우 boto regexp를 적절한 IAM 역할 arn 문자열로 바꿔주세요.

In [ ]:
bucket = '<your_s3_bucket_name_here>'
prefix = 'sagemaker/DEMO-linear-mnist'
 
# Define IAM role
import boto3
import re
from sagemaker import get_execution_role

role = get_execution_role()

### 데이터 수집

다음, 훈련 전에 전처리를 위해 온라인 URL에서 메모리로 데이터 세트를 읽습니다. 이 절차는 데이터 세트가 적절한 위치에 있다고 가정 할 때 Amazon Athena, Amazon EMR의 Apache Spark, Amazon Redshift 등*에서 수행* 할 수 있습니다. 다음 단계는 훈련에 사용하기 위해 데이터를 S3로 전송하는 것입니다. 큰 데이터 세트에 대해서는 메모리로 읽는 것이 번거롭겠지만 이와 같은 작은 데이터 세트에서는 그렇지 않습니다.


In [ ]:
%%time
import pickle, gzip, numpy, urllib.request, json

# Load the dataset
urllib.request.urlretrieve("http://deeplearning.net/data/mnist/mnist.pkl.gz", "mnist.pkl.gz")
with gzip.open('mnist.pkl.gz', 'rb') as f:
    train_set, valid_set, test_set = pickle.load(f, encoding='latin1')

### 데이터 검사

데이터 세트를 가져온 후에는 데이터를 검사하고 분포를 이해하며 필요한 전처리 유형을 결정하는 일반적인 머신 러닝 프로세스입니다. 이런 작업들은 바로 노트북에서 수행할 수 있습니다. 예를 들어, 데이터 세트의 일부인 숫자 중 하나를 살펴 보겠습니다.


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (2,10)


def show_digit(img, caption='', subplot=None):
    if subplot==None:
        _,(subplot)=plt.subplots(1,1)
    imgr=img.reshape((28,28))
    subplot.axis('off')
    subplot.imshow(imgr, cmap='gray')
    plt.title(caption)

show_digit(train_set[0][30], 'This is a {}'.format(train_set[1][30]))

### 데이터 변환

알고리즘에는 특정 입력 및 출력 요구 사항이 있으므로 데이터 세트 변환은 데이터 과학자가 훈련을 시작하기 전에 거치는 프로세스의 일부이기도합니다. 이 특별한 경우, Linear Learner의 Amazon SageMaker 구현은 recordIO-wrapped protobuf를 사용합니다. 오늘 사용할 데이터는 디스크에 있는 pickle-ized numpy 배열입니다.

대부분의 변환은 아래에서 import한 `sagemaker`, Amazon SageMaker Python SDK에 의해 처리됩니다.

In [ ]:
import io
import numpy as np
import sagemaker.amazon.common as smac

vectors = np.array([t.tolist() for t in train_set[0]]).astype('float32')
labels = np.where(np.array([t.tolist() for t in train_set[1]]) == 0, 1, 0).astype('float32')

buf = io.BytesIO()
smac.write_numpy_to_dense_tensor(buf, vectors, labels)
buf.seek(0)

## 훈련 데이터 업로드
RecordIO-wrapped protobuf를 생성하였고 이제 Amazon SageMaker 훈련에서 사용할 수 있도록 S3에 업로드해야합니다.


In [ ]:
import boto3
import os

key = 'recordio-pb-data'
boto3.resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'train', key)).upload_fileobj(buf)
s3_train_data = 's3://{}/{}/train/{}'.format(bucket, prefix, key)
print('uploaded training data location: {}'.format(s3_train_data))

Let's also setup an output S3 location for the model artifact that will be output as the result of training with the algorithm.

알고리즘 학습의 결과로 출력 될 모델 아티팩트에 대한 출력 S3 위치를 설정해봅시다.

In [ ]:
output_location = 's3://{}/{}/output'.format(bucket, prefix)
print('training artifacts will be uploaded to: {}'.format(output_location))

## 선형 모델 훈련

훈련을 위한 올바른 형식으로 데이터를 사전 처리가 완료되면 다음 단계로 실제로 데이터를 사용하여 모델을 훈련합니다. 테라 바이트 급 데이터 세트에서도 테스트했으나 이 데이터는 상대적으로 작기 때문에 Linear Learner 훈련 알고리즘의 성능을 표현하려고하는 것은 아닙니다.

Amazon SageMaker Python SDK를 사용하여 훈련을 시작하고 완료 될 때까지 상태를 모니터링합니다. 이 예제는 7 ~ 11분이 걸립니다. 데이터 세트가 작으나 하드웨어 프로비저닝 및 알고리즘 컨테이너를 로드하는데 시간이 걸립니다.

먼저 컨테이너를 지정합니다. 이 노트북이 4개의 Amazon SageMaker를 지원하는 리전에서 모두 실행되기를 원하므로 컨테이너 URI를 불러옵니다. 알고리즘 컨테이너에 대한 자세한 내용은 [AWS documentation](https://docs-aws.amazon.com/sagemaker/latest/dg/sagemaker-algo-docker-registry-paths.html)에서 확인할 수 있습니다.

In [ ]:
from sagemaker.amazon.amazon_estimator import get_image_uri
container = get_image_uri(boto3.Session().region_name, 'linear-learner')

다음 기본 estimator를 시작하여 필요한 하이퍼파라미터를 전달합니다.
비고:
- `feature_dim` 은 784로 설정되는데, 이는 28 x 28 이미지의 픽셀 수입니다.
- `predictor_type` 이미지가 0 인지 아닌지를 예측하려고하기 때문에 'binary_classifier'로 설정됩니다. 
- `mini_batch_size`는 200으로 설정됩니다. 이 값은 적합도 및 속도가 약간 향상되도록 조정할 수 있지만 대부분의 경우 데이터 세트에 의해 합리적인 값을 선택하는 것이 적합합니다.

In [ ]:
import boto3
import sagemaker

sess = sagemaker.Session()

linear = sagemaker.estimator.Estimator(container,
                                       role, 
                                       train_instance_count=1, 
                                       train_instance_type='ml.c4.xlarge',
                                       output_path=output_location,
                                       sagemaker_session=sess)
linear.set_hyperparameters(feature_dim=784,
                           predictor_type='binary_classifier',
                           mini_batch_size=200)

linear.fit({'train': s3_train_data})

## 모델 호스팅 설정
모델 훈련을 마쳤으므로 Amazon SageMaker 실시간 호스팅 엔트포인드 뒤에 모델을 배포할 수 있습니다. 이것은 모델에서 동적으로 예측 (또는 추론) 할수 있게합니다.

_Amazon SageMaker를 사용하면 다른 곳에서 학습한 모델을 유연하게 import 할 수 있으나 모델 생성 대상이 AWS Lambda, AWS Greengrass, Amazon Redshift, Amazon Athena 또는 기타 배포 대상인 경우 모델을 import 하지 않을 수 있습니다._

In [ ]:
linear_predictor = linear.deploy(initial_instance_count=1,
                                 instance_type='ml.m4.xlarge')

## 사용하기위한 모델 검증
마지막으로 모델의 유효성을 검사 할 수 있습니다. 예측 결과를 얻기 위해 HTTP POST 요청을 엔드포인트로 전달할 수 있습니다. 이를보다 쉽게하기 위해 Amazon SageMaker Python SDK를 다시 사용하고 알고리즘과 관련된 요청을 serialize하고 응답을 deserialize하는 방법을 지정합니다. 

In [ ]:
from sagemaker.predictor import csv_serializer, json_deserializer

linear_predictor.content_type = 'text/csv'
linear_predictor.serializer = csv_serializer
linear_predictor.deserializer = json_deserializer

이제 단일 레코드에 대한 예측을 진행해봅니다.

In [ ]:
result = linear_predictor.predict(train_set[0][30:31])
print(result)

단일 레코드에 대한 예측이 완료하였습니다. 하나의 레코드에 대해 엔드 포인트가`score` 및`predicted_label`을 포함하여`predictions`를 포함하는 일부 JSON을 리턴했음을 알 수 있습니다. 이 경우 `score`는 숫자가 0일 확률을 나타내는 [0, 1] 사이의 연속적인 값입니다. `predicted_label`은`0` 또는`1`의 값을 취합니다. 여기서 `1`은 (직관적으로) 이미지가 0임을 예측하고,`0`은 이미지가 0이 아니라고 예측하고 있음을 나타냅니다.

전체 이미지 배치를 수행하고 예측 정확도를 평가합시다.

In [ ]:
import numpy as np

predictions = []
for array in np.array_split(test_set[0], 100):
    result = linear_predictor.predict(array)
    predictions += [r['predicted_label'] for r in result['predictions']]

predictions = np.array(predictions)

In [ ]:
import pandas as pd

pd.crosstab(np.where(test_set[1] == 0, 1, 0), predictions, rownames=['actuals'], colnames=['predictions'])

위의 혼동 행렬에서 알 수 있듯이 931개 0 이미지를 올바르게 예측한 반면 44개 0이 아닌 이미지를 0으로 예측했고 49개 0인 이미지를 예측하지 못했습니다.  


### (선택 사항) 엔드포인트 삭제

이 노트북을 완료했다면 아래 셀에서 delete_endpoint 라인을 실행하세요. 생성된 호스팅 엔드포인트가 제거되고 남겨진 인스턴스에서 비용 발생하는 것을 방지할 수 있습니다.

In [ ]:
import sagemaker

sagemaker.Session().delete_endpoint(linear_predictor.endpoint)